In [1]:
# import importlib
# import local_utils; importlib.reload(local_utils)
from local_utils import *

/opt/conda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Seed: 7961730


In [2]:
raw_train_df = pd.read_csv(data_dir + 'train.csv', encoding='utf-8', low_memory=False, header=0,
                           names=['obs_id', 'site_id', 'timestamp', 'forecast_id', 'value'])
raw_train_df["timestamp"] = pd.to_datetime(raw_train_df["timestamp"])
raw_train_df.drop(columns='obs_id', inplace=True)
raw_train_df.sort_values(["timestamp", "site_id"], inplace=True)
raw_train_df["part"] = "train"

print(len(raw_train_df))
raw_train_df.head()

6559830


,site_id,timestamp,forecast_id,value,part
6218958,297,2009-06-23 06:45:00,6603,96408.826731,train
6218959,297,2009-06-23 07:45:00,6603,76800.251802,train
6218960,297,2009-06-23 08:45:00,6603,75166.203892,train
6218961,297,2009-06-23 09:45:00,6603,71898.108070,train
6218962,297,2009-06-23 10:45:00,6603,73532.155981,train


In [3]:
len(raw_train_df["forecast_id"].unique())

6974

In [4]:
raw_train_df.groupby("forecast_id").size().value_counts(dropna=False)

964    6750
360     121
90      103
dtype: int64

In [5]:
raw_test_df = pd.read_csv(data_dir + 'submission_format.csv', encoding='utf-8', low_memory=False, header=0,
                            names=['obs_id', 'site_id', 'timestamp', 'forecast_id', 'value'])
raw_test_df["timestamp"] = pd.to_datetime(raw_test_df["timestamp"])
raw_test_df.drop(columns='obs_id', inplace=True)
raw_test_df.sort_values(["timestamp", "site_id"], inplace=True)
raw_test_df["part"] = "test"

print(len(raw_test_df))
raw_test_df.head()

1309176


,site_id,timestamp,forecast_id,value,part
1240930,297,2009-08-02 10:45:00,6603,0.000000,test
1240931,297,2009-08-02 11:45:00,6603,0.000000,test
1240932,297,2009-08-02 12:45:00,6603,0.000000,test
1240933,297,2009-08-02 13:45:00,6603,0.000000,test
1240934,297,2009-08-02 14:45:00,6603,0.000000,test


In [6]:
raw_test_df.groupby("forecast_id").size().value_counts(dropna=False)

192    6661
60      135
191      89
58       86
59        3
dtype: int64

In [7]:
raw_df = pd.concat([raw_train_df, raw_test_df], ignore_index=True).sort_values(["site_id", "timestamp"], inplace=False)

print(len(raw_df))
raw_df.head()

7869006


,site_id,timestamp,forecast_id,value,part
953171,1,2014-09-03,1,909655.510515,train
956110,1,2014-09-04,1,1748273.124752,train
958853,1,2014-09-05,1,nan,train
961486,1,2014-09-06,1,nan,train
964345,1,2014-09-07,1,nan,train


In [8]:
raw_df.groupby("part").size()

part
test     1309176
train    6559830
dtype: int64

In [9]:
raw_df["time_diff"] = raw_df.groupby("forecast_id")["timestamp"].diff().fillna(method='bfill')
raw_df.groupby("time_diff").size()

time_diff
0 days 00:15:00    6293264
0 days 01:00:00    1509647
1 days 00:00:00      66095
dtype: int64

In [10]:
raw_df["date"] = pd.to_datetime(raw_df["timestamp"].dt.date)
raw_df["time"] = raw_df["timestamp"].dt.time
raw_df["week_day"] = raw_df["timestamp"].dt.weekday

raw_df["year"] = raw_df["timestamp"].dt.year
raw_df["month"] = raw_df["timestamp"].dt.month
raw_df["week"] = raw_df["timestamp"].dt.week
raw_df["day"] = raw_df["timestamp"].dt.day
raw_df["hour"] = raw_df["timestamp"].dt.hour
raw_df["minute"] = raw_df["timestamp"].dt.minute

In [11]:
raw_df.head()

,site_id,timestamp,forecast_id,value,part,time_diff,date,time,week_day,year,month,week,day,hour,minute
953171,1,2014-09-03,1,909655.510515,train,1 days,2014-09-03,00:00:00,2,2014,9,36,3,0,0
956110,1,2014-09-04,1,1748273.124752,train,1 days,2014-09-04,00:00:00,3,2014,9,36,4,0,0
958853,1,2014-09-05,1,nan,train,1 days,2014-09-05,00:00:00,4,2014,9,36,5,0,0
961486,1,2014-09-06,1,nan,train,1 days,2014-09-06,00:00:00,5,2014,9,36,6,0,0
964345,1,2014-09-07,1,nan,train,1 days,2014-09-07,00:00:00,6,2014,9,36,7,0,0


In [12]:
metadata_df = pd.read_csv(data_dir + 'metadata.csv', encoding='utf-8', low_memory=False, header=0,
                          names=['site_id', 'surface', 'sampling', 'base_temperature', 
                                 'mon_do', 'tue_do', 'wed_do', 'thu_do', 'fri_do', 'sat_do', 'sun_do'])
metadata_df.head()

,site_id,surface,sampling,base_temperature,mon_do,tue_do,wed_do,thu_do,fri_do,sat_do,sun_do
0,1,1387.205119,15.000000,18.000000,False,False,False,False,False,True,True
1,2,6098.278376,30.000000,18.000000,False,False,False,False,False,True,True
2,3,10556.293605,5.000000,18.000000,False,False,False,False,False,True,False
3,5,12541.181277,30.000000,18.000000,False,False,False,False,False,True,True
4,6,9150.195373,30.000000,18.000000,False,False,False,False,False,True,True


In [13]:
merged_df = raw_df.merge(metadata_df, how='left', on='site_id')
merged_df.head()

,site_id,timestamp,forecast_id,value,part,time_diff,date,time,week_day,year,...,surface,sampling,base_temperature,mon_do,tue_do,wed_do,thu_do,fri_do,sat_do,sun_do
0,1,2014-09-03,1,909655.510515,train,1 days,2014-09-03,00:00:00,2,2014,...,1387.205119,15.000000,18.000000,False,False,False,False,False,True,True
1,1,2014-09-04,1,1748273.124752,train,1 days,2014-09-04,00:00:00,3,2014,...,1387.205119,15.000000,18.000000,False,False,False,False,False,True,True
2,1,2014-09-05,1,nan,train,1 days,2014-09-05,00:00:00,4,2014,...,1387.205119,15.000000,18.000000,False,False,False,False,False,True,True
3,1,2014-09-06,1,nan,train,1 days,2014-09-06,00:00:00,5,2014,...,1387.205119,15.000000,18.000000,False,False,False,False,False,True,True
4,1,2014-09-07,1,nan,train,1 days,2014-09-07,00:00:00,6,2014,...,1387.205119,15.000000,18.000000,False,False,False,False,False,True,True


In [14]:
days_off_cols = ["mon_do", "tue_do", "wed_do", "thu_do", "fri_do", "sat_do", "sun_do"]
merged_df["is_day_off"] = (((merged_df[days_off_cols[4]].values)&(merged_df["week_day"] == 4))|
                           ((merged_df[days_off_cols[5]].values)&(merged_df["week_day"] == 5))|
                           ((merged_df[days_off_cols[6]].values)&(merged_df["week_day"] == 6)))
merged_df.drop(columns=days_off_cols, inplace=True)

In [15]:
holidays_df = pd.read_csv(data_dir + 'holidays.csv', encoding='utf-8', low_memory=False, header=0,
                          names=['row_id', 'date', 'holiday', 'site_id'])
holidays_df["date"] = pd.to_datetime(holidays_df["date"])
holidays_df.sort_values(["date", "site_id"], inplace=True)
holidays_df.drop_duplicates(["date", "site_id"], inplace=True)
holidays_df.drop(columns="row_id", inplace=True)

holidays_df.head()

,date,holiday,site_id
7572,2009-01-01,New year,290
7810,2009-01-01,New year,297
8021,2009-01-01,New year,301
8129,2009-01-01,New year,302
8237,2009-01-01,New year,303


In [16]:
merged_df = merged_df.merge(holidays_df, how='left', on=['site_id', 'date'])
merged_df["is_holiday"] = merged_df["holiday"].notnull()
merged_df["holiday"].fillna("not a holiday", inplace=True)
merged_df.head()

,site_id,timestamp,forecast_id,value,part,time_diff,date,time,week_day,year,...,week,day,hour,minute,surface,sampling,base_temperature,is_day_off,holiday,is_holiday
0,1,2014-09-03,1,909655.510515,train,1 days,2014-09-03,00:00:00,2,2014,...,36,3,0,0,1387.205119,15.000000,18.000000,False,not a holiday,False
1,1,2014-09-04,1,1748273.124752,train,1 days,2014-09-04,00:00:00,3,2014,...,36,4,0,0,1387.205119,15.000000,18.000000,False,not a holiday,False
2,1,2014-09-05,1,nan,train,1 days,2014-09-05,00:00:00,4,2014,...,36,5,0,0,1387.205119,15.000000,18.000000,False,not a holiday,False
3,1,2014-09-06,1,nan,train,1 days,2014-09-06,00:00:00,5,2014,...,36,6,0,0,1387.205119,15.000000,18.000000,True,not a holiday,False
4,1,2014-09-07,1,nan,train,1 days,2014-09-07,00:00:00,6,2014,...,36,7,0,0,1387.205119,15.000000,18.000000,True,not a holiday,False


In [17]:
weather_df = pd.read_csv(data_dir + 'weather.csv', encoding='utf-8', low_memory=False, header=0,
                         names=['row_id', 'timestamp', 'temperature', 'distance', 'site_id'])
weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])
weather_df.drop(columns='row_id', inplace=True)
weather_df.sort_values(["site_id", "timestamp"], inplace=True)
weather_df.head()

,timestamp,temperature,distance,site_id
0,2013-12-31 19:00:00,-7.200000,24.889929,1
1,2013-12-31 19:00:00,-8.300000,23.303097,1
2,2013-12-31 19:00:00,-7.800000,20.952256,1
3,2013-12-31 19:00:00,-8.000000,20.952256,1
4,2013-12-31 19:34:00,-8.100000,16.610602,1


In [18]:
closest_weather_df = weather_df\
    .sort_values(["site_id", "timestamp", "distance"], inplace=False)\
    .drop_duplicates(["site_id", "timestamp"], inplace=False)[["site_id", "timestamp", "distance", "temperature"]]\
    .reset_index(drop=True)\
    .rename(columns={"distance": "closest_distance", "temperature": "closest_temperature"})
closest_weather_df.head()

,site_id,timestamp,closest_distance,closest_temperature
0,1,2013-12-31 19:00:00,20.952256,-7.800000
1,1,2013-12-31 19:34:00,16.610602,-8.100000
2,1,2013-12-31 19:55:00,16.610602,-8.200000
3,1,2013-12-31 20:00:00,20.952256,-8.000000
4,1,2013-12-31 20:15:00,16.610602,-8.300000


In [19]:
avg_weather_df = weather_df\
    .groupby(["site_id", "timestamp"])[["distance", "temperature"]]\
    .mean()\
    .reset_index()\
    .rename(columns={"distance": "avg_distance", "temperature": "avg_temperature"})
avg_weather_df.head()

,site_id,timestamp,avg_distance,avg_temperature
0,1,2013-12-31 19:00:00,22.524384,-7.825000
1,1,2013-12-31 19:34:00,16.610602,-8.100000
2,1,2013-12-31 19:55:00,16.610602,-8.200000
3,1,2013-12-31 20:00:00,22.921092,-7.900000
4,1,2013-12-31 20:15:00,16.610602,-8.300000


In [20]:
avg_day_weather_df = weather_df
avg_day_weather_df["date"] = pd.to_datetime(weather_df["timestamp"].dt.date)
avg_day_weather_df = avg_day_weather_df\
    .groupby(["site_id", "date"])[["distance", "temperature"]]\
    .mean()\
    .reset_index()\
    .rename(columns={"distance": "avg_day_distance", "temperature": "avg_day_temperature"})
avg_day_weather_df.head()

,site_id,date,avg_day_distance,avg_day_temperature
0,1,2013-12-31,19.719508,-8.376923
1,1,2014-01-01,22.935144,-9.895652
2,1,2014-01-02,22.951963,-10.917722
3,1,2014-01-03,22.840771,-13.950000
4,1,2014-01-04,22.840771,-5.281667


In [21]:
avg_month_weather_df = weather_df
avg_month_weather_df["month"] = weather_df["timestamp"].dt.month
avg_month_weather_df = avg_month_weather_df\
    .groupby(["site_id", "month"])[["distance", "temperature"]]\
    .mean()\
    .reset_index()\
    .rename(columns={"distance": "avg_month_distance", "temperature": "avg_month_temperature"})
avg_month_weather_df.head()

,site_id,month,avg_month_distance,avg_month_temperature
0,1,1,22.576659,-4.069527
1,1,2,22.457980,-3.836865
2,1,3,22.590272,1.414287
3,1,4,22.398653,8.783595
4,1,5,22.643451,15.349275


In [22]:
merged_df.sort_values(["timestamp", "site_id"], inplace=True)
closest_weather_df.sort_values(["timestamp", "site_id"], inplace=True)
avg_weather_df.sort_values(["timestamp", "site_id"], inplace=True)
avg_day_weather_df.sort_values(["date", "site_id"], inplace=True)
avg_month_weather_df.sort_values(["month", "site_id"], inplace=True)

merged_df = pd.merge_asof(merged_df, closest_weather_df, by="site_id", on="timestamp", direction="nearest")
merged_df = pd.merge_asof(merged_df, avg_weather_df, by="site_id", on="timestamp", direction="nearest")
merged_df = pd.merge_asof(merged_df, avg_day_weather_df, by="site_id", on="date", direction="nearest")
merged_df = merged_df.merge(avg_month_weather_df, how="left", on=["site_id", "month"])

merged_df.sort_values(["site_id", "timestamp"], inplace=True)
merged_df["has_weather"] = merged_df["closest_temperature"].notnull()
merged_df["closest_distance"].fillna(0, inplace=True)
merged_df["closest_temperature"].fillna(merged_df["base_temperature"], inplace=True)
merged_df["avg_distance"].fillna(0, inplace=True)
merged_df["avg_temperature"].fillna(merged_df["base_temperature"], inplace=True)
merged_df["avg_day_distance"].fillna(0, inplace=True)
merged_df["avg_day_temperature"].fillna(merged_df["base_temperature"], inplace=True)
merged_df["avg_month_distance"].fillna(0, inplace=True)
merged_df["avg_month_temperature"].fillna(merged_df["base_temperature"], inplace=True)
merged_df["closest_temperature_base_diff"] = np.abs(merged_df["base_temperature"] - merged_df["closest_temperature"])
merged_df["avg_temperature_base_diff"] = np.abs(merged_df["base_temperature"] - merged_df["avg_temperature"])
merged_df["avg_day_temperature_base_diff"] = np.abs(merged_df["base_temperature"] - merged_df["avg_day_temperature"])
merged_df["avg_month_temperature_base_diff"] = np.abs(merged_df["base_temperature"] - merged_df["avg_month_temperature"])
merged_df.head()

,site_id,timestamp,forecast_id,value,part,time_diff,date,time,week_day,year,...,is_holiday,closest_distance,closest_temperature,avg_distance,avg_temperature,avg_day_distance,avg_day_temperature,avg_month_distance,avg_month_temperature,has_weather
1138708,1,2014-09-03,1,909655.510515,train,1 days,2014-09-03,00:00:00,2,2014,...,False,20.952256,19.000000,22.921092,19.600000,22.872779,21.184746,22.418264,18.695938,True
1141835,1,2014-09-04,1,1748273.124752,train,1 days,2014-09-04,00:00:00,3,2014,...,False,20.952256,20.000000,22.921092,21.300000,22.840771,23.375000,22.418264,18.695938,True
1144962,1,2014-09-05,1,nan,train,1 days,2014-09-05,00:00:00,4,2014,...,False,20.952256,22.000000,22.921092,23.350000,22.827057,26.521667,22.418264,18.695938,True
1148089,1,2014-09-06,1,nan,train,1 days,2014-09-06,00:00:00,5,2014,...,False,20.952256,21.000000,22.921092,21.600000,22.872779,19.944068,22.418264,18.695938,True
1151216,1,2014-09-07,1,nan,train,1 days,2014-09-07,00:00:00,6,2014,...,False,20.952256,13.000000,22.921092,15.800000,22.840771,17.680000,22.418264,18.695938,True


In [28]:
merged_df.isnull().sum()

site_id                                0
timestamp                              0
forecast_id                            0
value                              86601
part                                   0
time_diff                              0
date                                   0
time                                   0
week_day                               0
year                                   0
month                                  0
week                                   0
day                                    0
hour                                   0
minute                                 0
surface                                0
sampling                               0
base_temperature                       0
is_day_off                             0
holiday                                0
is_holiday                             0
closest_distance                       0
closest_temperature                    0
avg_distance                           0
avg_temperature 

In [29]:
merged_df.to_hdf(data_dir+"prepared_data.hdf", key="key_0", mode="w", format="f")

/opt/conda/lib/python3.5/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->['part', 'time', 'holiday']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [ ]:
stop

In [40]:
# shifts = []
# for shift in range(1,4):
#     shifts.append(
#         merged_df\
#             .groupby("forecast_id")[["closest_distance", "closest_temperature", "avg_distance", "avg_temperature"]]\
#             .shift(-shift)\
#             .rename(columns={"closest_distance": "closest_distance_"+str(shift), 
#                              "closest_temperature": "closest_temperature_"+str(shift), 
#                              "avg_distance": "avg_distance_"+str(shift), 
#                              "avg_temperature": "avg_temperature_"+str(shift)}))
    
# weather_shifts_df = pd.concat(shifts, axis=1, ignore_index=True)

In [27]:
submission_df = pd.read_csv(data_dir + 'submission_format.csv', encoding='utf-8', low_memory=False, header=0,
                            names=['obs_id', 'site_id', 'timestamp', 'forecast_id', 'value'])
submission_df["timestamp"] = pd.to_datetime(submission_df["timestamp"])

# submission_freq_df = pd.read_csv(data_dir + 'submission_frequency.csv', encoding='utf-8', low_memory=False, header=0,
#                             names=['forecast_id', 'forecast_period'])
# submission_freq_df["forecast_period"] = pd.to_datetime(submission_freq_df["forecast_period"]).dt.time

submission_df.groupby(["forecast_id"]).size()

forecast_id
1        60
2        60
3        60
4        58
5       192
6       192
7       192
8       192
9       192
10      192
11      192
12      192
13      192
14      192
15      192
16      192
17      192
18      192
19      192
20      192
21      192
22      192
23      192
24      192
25      192
26      192
27      192
28      192
29      192
30      192
       ... 
6945    192
6946    192
6947    192
6948    192
6949    192
6950    192
6951    192
6952    192
6953    192
6954    192
6955    192
6956    192
6957    192
6958    192
6959    192
6960    192
6961    192
6962     60
6963     60
6964     60
6965     60
6966     60
6967     60
6968     60
6969     60
6970     60
6971     58
6972     60
6973     58
6974    192
Length: 6974, dtype: int64